In [1]:
import numpy as np
from PIL import Image
from scipy import ndimage
import matplotlib.pyplot as plt

def load_image(path):
    """Load image and convert to grayscale numpy array"""
    img = Image.open(path).convert('L')
    return np.array(img)

def gradient_magnitude(img):
    """Calculate gradient magnitude using Sobel operators"""
    # Sobel operators
    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]])
    
    sobel_y = np.array([[-1, -2, -1],
                        [0, 0, 0],
                        [1, 2, 1]])
    
    # Calculate gradients
    grad_x = ndimage.convolve(img.astype('float32'), sobel_x)
    grad_y = ndimage.convolve(img.astype('float32'), sobel_y)
    
    # Calculate magnitude
    gradient = np.sqrt(grad_x**2 + grad_y**2)
    return gradient

def find_local_minima(gradient_img):
    """Find local minima in the gradient image"""
    neighborhood = ndimage.generate_binary_structure(2, 2)
    local_min = (gradient_img == ndimage.minimum_filter(gradient_img, 
                                                    footprint=neighborhood))
    return local_min

def label_markers(local_min):
    """Label the markers for watershed"""
    markers, num_features = ndimage.label(local_min)
    return markers

def watershed_transform(gradient_img, markers):
    """Implement watershed algorithm"""
    # Get image dimensions
    height, width = gradient_img.shape
    
    # Create a queue for pixels to process
    queue = []
    
    # Initialize watershed image
    watershed = markers.copy()
    
    # Find initial boundary pixels
    for i in range(1, height-1):
        for j in range(1, width-1):
            if watershed[i,j] != 0:
                # Check neighbors
                for di in [-1, 0, 1]:
                    for dj in [-1, 0, 1]:
                        if watershed[i+di,j+dj] == 0:
                            queue.append((gradient_img[i,j], i, j))
                            break
    
    # Sort queue by gradient magnitude
    queue.sort()
    
    # Process pixels
    while queue:
        _, i, j = queue.pop(0)
        
        # Get neighboring labels
        neighbor_labels = set()
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                if (di != 0 or dj != 0) and watershed[i+di,j+dj] != 0:
                    neighbor_labels.add(watershed[i+di,j+dj])
        
        # Assign label if all non-zero neighbors have same label
        if len(neighbor_labels) == 1:
            watershed[i,j] = neighbor_labels.pop()
            
            # Add neighbors to queue
            for di in [-1, 0, 1]:
                for dj in [-1, 0, 1]:
                    if watershed[i+di,j+dj] == 0:
                        queue.append((gradient_img[i+di,j+dj], i+di, j+dj))
            queue.sort()
    
    return watershed

def apply_watershed(image_path):
    """Apply complete watershed segmentation pipeline"""
    # Load and preprocess image
    img = load_image(image_path)
    
    # Calculate gradient magnitude
    gradient = gradient_magnitude(img)
    
    # Find local minima
    local_min = find_local_minima(gradient)
    
    # Label markers
    markers = label_markers(local_min)
    
    # Apply watershed transform
    result = watershed_transform(gradient, markers)
    
    return img, result

def display_results(original, watershed_result):
    """Display original image and watershed result"""
    plt.figure(figsize=(12, 4))
    
    plt.subplot(131)
    plt.imshow(original, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    
    plt.subplot(132)
    plt.imshow(watershed_result, cmap='nipy_spectral')
    plt.title('Watershed Segmentation')
    plt.axis('off')
    
    plt.subplot(133)
    plt.imshow(original, cmap='gray')
    plt.imshow(watershed_result, cmap='nipy_spectral', alpha=0.3)
    plt.title('Overlay')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    # Example usage
    image_path = "./Lab 1.jpg"  # Replace with your image path
    original_img, result = apply_watershed(image_path)
    display_results(original_img, result)

IndexError: index 400 is out of bounds for axis 1 with size 400